In [43]:
import numpy as np
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
import time

def read_tsp(file_path):
    """ TSP LIB 形式のファイルから頂点情報を読み取る """
    nodes = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    is_coord_section = False
    for line in lines:
        line = line.strip()
        if line.startswith("NODE_COORD_SECTION"):
            is_coord_section = True
            continue
        if line.startswith("EOF"):
            break
        if is_coord_section:
            parts = line.split()
            if len(parts) >= 3:
                node_id = int(parts[0])
                x, y = float(parts[1]), float(parts[2])
                nodes[node_id] = (x, y)
    return nodes

def read_tour(file_path):
    """ ツアーファイルを読み込む（単純なIDリスト） """
    tour = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.isdigit():
                tour.append(int(line))
    return tour

def extract_unique_nodes(nodes1, nodes2):
    """ 2つのノードセットから片方にしか存在しないノードを抽出 """
    set1 = set(nodes1.keys())
    set2 = set(nodes2.keys())
    
    unique1 = {k: nodes1[k] for k in set1 - set2}
    unique2 = {k: nodes2[k] for k in set2 - set1}
    
    return unique1, unique2

def solve_assignment(unique1, unique2):
    """ 最小重み完全マッチングをハンガリー法で解く """
    ids1, coords1 = zip(*unique1.items()) if unique1 else ([], [])
    ids2, coords2 = zip(*unique2.items()) if unique2 else ([], [])
    
    if not coords1 or not coords2:
        return {}  # 一方が空ならマッチング不可
    
    cost_matrix = distance_matrix(coords1, coords2)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    matches = {ids1[i]: ids2[j] for i, j in zip(row_ind, col_ind)}
    return matches

def update_tour(tour, matches):
    """ 既存のツアーをマッチング結果で更新する """
    return [matches.get(node, node) for node in tour]

def write_tour(file_path, tour):
    """ 更新したツアー情報をファイルに出力する """
    with open(file_path, 'w') as f:
        for node in tour:
            f.write(f"{node}\n")
        f.write("-1\n")

def main(tsp_file1, tsp_file2, tour_file, output_tour_file):
    start_time = time.time()
    nodes1 = read_tsp(tsp_file1)
    nodes2 = read_tsp(tsp_file2)
    tour = read_tour(tour_file)
    
    unique1, unique2 = extract_unique_nodes(nodes1, nodes2)
    matches = solve_assignment(unique1, unique2)
    
    updated_tour = update_tour(tour, matches)
    
    write_tour(output_tour_file, updated_tour)
    end_time = time.time()
    print(f"Updated tour written to {output_tour_file}")
    print(f"Execution time: {end_time - start_time:.6f} seconds")

# 使用例
tsp_file1 = "original_90k_n5.tsp"
tsp_file2 = "similar_90K_n5.tsp"
tour_file = "original_result_90k_n5_3opt.tour"
output_tour_file = "updated_initial_90k_n5_3opt.tour"

main(tsp_file1, tsp_file2, tour_file, output_tour_file)



Updated tour written to updated_initial_90k_n5_3opt.tour
Execution time: 0.223386 seconds
